In [ ]:
var2mapeig = np.zeros ((testpointsj_l_scatter, testpointsj_g_scatter))
map2realeig = [[ ] for i in range(testpointsj_l_scatter)]
map2imageig = [[ ] for i in range(testpointsj_l_scatter)]

# Map for IKH condition
map2wpeig_char   = [[ ] for i in range(testpointsj_l_scatter)]

# Map for VKH condition
map2wpeig   = [[ ] for i in range(testpointsj_l_scatter)]

ang_freq_imag = np.zeros ((testpointsbeta,testpointsj_l_scatter, testpointsj_g_scatter))
ang_freq_real = np.zeros ((testpointsbeta,testpointsj_l_scatter, testpointsj_g_scatter))

eig_vect = ThreeD(testpointsbeta,testpointsj_l_scatter, testpointsj_g_scatter)

k = 0
for beta in betavector:
    print (" >>>  = ", np.rad2deg(beta), "deg")
    j = 0 #for each element of j2vector
    for j_g in j_gscatter: #iterative process for j2
        print (">>j_g = ", j_g)
        i = 0 #for each element of j1vector
        
        # listj_l_imageig = [[] for i in range(6)] ######nao usado
        # listj_l_realeig = [[] for i in range(6)] ######nao usado
        
        for j_l in j_lscatter: #iterative process for j1
            print (">j_l = ", j_l)
            
            # finding the solution with fsolve
            var4 = var4_0
            var1eq = brenth (equilibrium1, lima, limb)
            print("var1eq = ", var1eq)
            var2mapeig [i,j] = var1eq
            
            # parameters used in boundary conditions
            var1_0 = var1eq
            var2_0 = var2 (var1eq)
            var3_0 = var3 (var1eq)
            
            Aeval = np.asarray (Amat (var1_0, var2_0, var3_0, var4_0))
            Beval = np.asarray (Bmat (var1_0, var2_0, var3_0, var4_0))
            Ceval = np.asarray (Cmat_lin (var1_0, var2_0, var3_0, var4_0))
            
            # Characteristics
            _, listimageig2char, _ = characteristics_function (Aeval, Beval)

            if all ([ v == 0 for v in listimageig2char]) :
                condition_char = 'IKH stable'
            else: 
                condition_char = 'IKH unstable'

            map2wpeig_char[i] = condition_char

            # Eigenvalues
            listrealeig2, listimageig2, _, v2eig = stability_function (Aeval, Beval, Ceval) 

            # Acomplex   = Aeval.dot (1j)
            # Bcomplex   = Beval.dot (1j)

            # wavenumber = wavenumber_fourier
            # meig2, v2eig = eig ( (-wavenumber_fourier*Bcomplex - Ceval), b =  -Acomplex, overwrite_a = True, overwrite_b = True, check_finite = True) 

            # listrealeig2 = []
            # listimageig2 = []
            
            printeigenvectors = []

            # for l in range (len (meig2)):
            #     realpart = meig2 [l].real                
            #     imagpart = meig2 [l].imag
                
            #     listimageig2.append (imagpart)
            #     listrealeig2.append (realpart)
            
            map2imageig[i] = np.array(listimageig2)
            map2realeig[i] = np.array(listrealeig2)

            if all ([ v > 0 for v in listimageig2]) :
                condition = 'VKH stable'
            else:
                # var2mapeig [k, j] = j_l
                condition = 'VKH unstable'

            map2wpeig[i] = condition

            print("imaginary angular frequencies", map2imageig[i])
            print("real angular frequencies", map2realeig[i])

            local1 = np.where(map2imageig[i] < 0)
            
            if any ([ vv < 0 for vv in map2imageig[i]]) :
                print("VKH unstable")
                print("unstable mode", map2imageig[i][local1])
                ang_freq_imag [k, j, i] = map2imageig[i][local1]
                print("imaginary part", ang_freq_imag)
                ang_freq_real [k, j, i] = map2realeig[i][local1]
                print("real part", ang_freq_real)
                print("eigenvectors", v2eig[local1][0])
                eig_vect [k][j][i] = v2eig[local1][0]
            else: 
                print("VKH stable")
                print("unstable mode", 0)
                ang_freq_imag [k, j, i] = 0
                print(ang_freq_imag)
                print("eigenvectors", 0)
                eig_vect [k][j][i] = 0
        
            i += 1      
        j += 1

    # Plot eigenspectra
    fig, ax = plt.subplots ()

    print("var2mapeig =", var2mapeig)
    for jj in range(testpointsj_l_scatter):
        ax.scatter (map2realeig [jj],
                    -map2imageig [jj], 
                    s = areatestpoints,
                    marker = listmarkers[jj],
                    color = listcolor [0], 
                    edgecolors = listcolor [0],
                    linewidths = 1,
                    alpha = alphascatter,
                    label = '$u_{sl}$'+ '= ' + str("%.3f" %j_lscatter [jj])+ ' $\it{m \; s^{-1}}$' + ', ' + '$u_{sg}$' + '= ' + str("%.3f" %j_gscatter[jj]) + ' $\it{m \; s^{-1}}$' + '$ , $' + r'$\alpha_l$' + '= ' + str("%.3f" %var2mapeig [jj, jj]) + ', ' \
                        + map2wpeig[jj] + ', ' + map2wpeig_char[jj] + '.') # r'$\beta$' + '= ' + str("%.2f" %betavectordeg [k]) + ' $\it{deg}$' + ', '

        # ax.set_xscale ('symlog')
        ax.xaxis.set_minor_locator(MultipleLocator(500))

        # plt.rcParams ['figure.figsize'] = mapsize
        leg1 = ax.legend (loc = 'upper right', 
                            frameon = True, 
                            fontsize = label_size-3, 
                            shadow = True)

        plt.grid (True, which = "both")
        
        matplotlib.rc ('xtick', labelsize = label_size)     
        matplotlib.rc ('ytick', labelsize = label_size)

        plt.ylim ( -12, 4)
        # plt.xlim (( -1e3, 1e3))
        
        ax.xaxis.set_tick_params(which='major', direction='in', top='on')
        ax.xaxis.set_tick_params(which='minor', direction='in', top='on')
        ax.yaxis.set_tick_params(which='major', direction='in', right='on')
        ax.yaxis.set_tick_params(which='minor', direction='in', right='on')

        ax.set_xlabel(r'Re ($\omega$) $[\it{s^{-1}}]$', fontsize = label_size)
        ax.set_ylabel(r'-Im ($\omega$) $[\it{s^{-1}}]$', fontsize = label_size)

        # Save figure
        fig.set_size_inches (mapsize)
        plt.savefig('results/figures/diff_maps/fig7_'+ str(k) +'.pdf',
                optimize = True,
                transparent = True,  
                dpi = dpi_elsevier)
    plt.show ()
    
    k += 1